In [1]:
import numpy as np
import gurobipy as gp
import torch

from graph import Graph
from bssrp_mip import BSSRPMIP
from nn_heuristic import NearestNeighboursHeuristic

from environment import Environment

In [2]:
tol = 1e-3

In [3]:
def demand_of_routes(routes, demands):
    for i in range(len(routes)):
        route = routes[i]
        route_demand_order = list(map(lambda x: demands[x], route))
        print(f"Vehicle {i}:")
        print("    Route: ", route)
        print("    Demand:", route_demand_order)

In [4]:
def eval_mip_sol_in_env(mip, g):
    graph_dict = {0 : g}
    env = Environment(graph_dict, "test")
    
    env.reset(0)

    route_reward = {}
    for i, route in mip.routes.items():
        route_reward[i] = 0
        if len(route) == 0:
            continue

        for a in route[1:]:
            _, reward, done, _ = env.step(torch.tensor(a))
            route_reward[i] += reward
            if done:
                break
        if done:
            break

    total_reward = 0
    for i, reward in route_reward.items():
        total_reward += reward
        
    assert(np.abs(mip.model.objVal + total_reward) < tol)
        
    return total_reward.item()

In [5]:
def eval_nn_in_env(nn, g):
    graph_dict = {0 : g}
    env = Environment(graph_dict, "test")
    
    env.reset(0)
    
    route_reward = {}
    for i, route in enumerate(nn.routes):
        route_reward[i] = 0
        if len(route) == 0:
            continue

        for a in route[1:]:
            _, reward, done, _ = env.step(torch.tensor(a))
            route_reward[i] += reward
            if done:
                break
        if done:
            break

    total_reward = 0
    for i, reward in route_reward.items():
        total_reward += reward
        
    return total_reward.item()

In [6]:
def evaluate(g, n_instances, seed):
    
    g.seed(seed)
    results = {
        "mip" : {
            "routes" : [],
            "cost" : [],
        },
        "nn" : {
            "routes" : [],
            "cost" : [],
        },
        "rl" : {
            "routes" : [],
            "cost" : [],
        }
    }
    
    for i in range(n_instances):
        
        g.bss_graph_gen()
        
        # 
        mip = BSSRPMIP(g, use_penalties=True, no_bikes_leaving=True)
        mip.model.setParam('OutputFlag', 0)
        mip.optimize()
        mip.optimize()
        mip_routes = list(mip.routes.values())
        mip_reward = eval_mip_sol_in_env(mip, g)
        results["mip"]["routes"].append(mip_routes)
        results["mip"]["cost"].append(mip_reward)
        
        # get NN of routes
        nn = NearestNeighboursHeuristic(g)
        nn_routes = nn.run()
        nn_reward = eval_nn_in_env(nn, g)
        results["nn"]["routes"].append(nn_routes)
        results["nn"]["cost"].append(nn_reward)
        
        # TODO: collect results for RL model
        
        
        
        
    return results
    

### Generate Graph

In [7]:
seed = 12343

In [8]:
use_penalties = True
no_bikes_leaving = True

In [9]:
num_nodes = 10
num_vehicles = 5
time_limit = 30

penalty_cost_demand = 2
penalty_cost_time = 5
bike_load_time = 0

speed = 30 

In [10]:
g = Graph(
        num_nodes = num_nodes, 
        k_nn = 2, 
        num_vehicles = num_vehicles,
        penalty_cost_demand = penalty_cost_demand,
        penalty_cost_time = penalty_cost_time, 
        speed = speed,
        bike_load_time=bike_load_time,
        time_limit = time_limit)

### Evaluate

In [11]:
n_instances = 100

In [12]:
results = evaluate(g, n_instances, seed)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-01-23
Set parameter TimeLimit to value 100000
Tour:  [0, 1, 3, 9, 0, 2, 5, 6, 7, 4, 8, 0]
Tour Time:  83.84409916710437
Left Demand:  tensor(0.)
Node Visits:  12
Games Finished:  0
####################################################################################################
Tour:  [0, 1, 2, 5, 0, 3, 6, 0, 7, 0, 9, 8, 0, 4, 0]
Tour Time:  190.19088433251957
Left Demand:  tensor(-5.)
Node Visits:  15
Games Finished:  0
####################################################################################################
Set parameter TimeLimit to value 100000
Tour:  [0, 6, 9, 1, 7, 0, 8, 3, 0, 4, 5, 2, 0]
Tour Time:  73.3780353302785
Left Demand:  tensor(-2.)
Node Visits:  13
Games Finished:  0
####################################################################################################
Tour:  [0, 8, 3, 9, 6, 0, 4, 5, 2, 0, 7, 1, 0]
Tour Time:  76.45091938640628
Left Demand:  tensor(-5.)
Node

Tour:  [0, 2, 4, 9, 3, 6, 1, 8, 0, 7, 5, 0]
Tour Time:  68.9610859228207
Left Demand:  tensor(0.)
Node Visits:  12
Games Finished:  0
####################################################################################################
Tour:  [0, 7, 5, 0, 4, 2, 9, 0, 3, 0, 8, 1, 6, 0]
Tour Time:  127.04172578434354
Left Demand:  tensor(-6.)
Node Visits:  14
Games Finished:  0
####################################################################################################
Set parameter TimeLimit to value 100000
Tour:  [0, 2, 8, 6, 4, 1, 0, 3, 9, 5, 0, 7, 0]
Tour Time:  70.12847454065201
Left Demand:  tensor(-3.)
Node Visits:  13
Games Finished:  0
####################################################################################################
Tour:  [0, 2, 8, 6, 1, 4, 0, 3, 9, 7, 0, 5, 0]
Tour Time:  88.00326628917203
Left Demand:  tensor(-8.)
Node Visits:  13
Games Finished:  0
####################################################################################################
S

Tour:  [0, 1, 6, 8, 0, 5, 7, 2, 0, 9, 3, 4, 0]
Tour Time:  82.82067549013017
Left Demand:  tensor(0.)
Node Visits:  13
Games Finished:  0
####################################################################################################
Tour:  [0, 5, 2, 1, 0, 3, 4, 9, 0, 6, 8, 0, 7, 0]
Tour Time:  118.205954537557
Left Demand:  tensor(-4.)
Node Visits:  14
Games Finished:  0
####################################################################################################
Set parameter TimeLimit to value 100000
Tour:  [0, 1, 9, 8, 7, 2, 4, 0, 6, 0, 3, 5, 0]
Tour Time:  89.15287209235198
Left Demand:  tensor(-2.)
Node Visits:  13
Games Finished:  0
####################################################################################################
Tour:  [0, 1, 9, 7, 8, 2, 3, 0, 4, 6, 5, 0]
Tour Time:  86.45077556239634
Left Demand:  tensor(-4.)
Node Visits:  12
Games Finished:  0
####################################################################################################
Se

Tour:  [0, 1, 7, 0, 2, 9, 0, 3, 5, 8, 6, 4, 0]
Tour Time:  87.28157452305342
Left Demand:  tensor(-2.)
Node Visits:  13
Games Finished:  0
####################################################################################################
Tour:  [0, 7, 1, 8, 0, 2, 5, 0, 3, 0, 4, 9, 6, 0]
Tour Time:  143.67209204371213
Left Demand:  tensor(-4.)
Node Visits:  14
Games Finished:  0
####################################################################################################
Set parameter TimeLimit to value 100000
Tour:  [0, 1, 6, 2, 5, 9, 0, 3, 8, 7, 4, 0]
Tour Time:  63.94297802869163
Left Demand:  tensor(0.)
Node Visits:  12
Games Finished:  0
####################################################################################################
Tour:  [0, 1, 6, 2, 9, 0, 3, 8, 7, 4, 5, 0]
Tour Time:  81.84586996793372
Left Demand:  tensor(-6.)
Node Visits:  12
Games Finished:  0
####################################################################################################
Set

Tour:  [0, 4, 8, 1, 7, 3, 2, 0, 5, 9, 6, 0]
Tour Time:  63.67251626221827
Left Demand:  tensor(0.)
Node Visits:  12
Games Finished:  0
####################################################################################################
Tour:  [0, 4, 8, 2, 3, 1, 0, 5, 9, 6, 7, 0]
Tour Time:  80.60610827566728
Left Demand:  tensor(-3.)
Node Visits:  12
Games Finished:  0
####################################################################################################
Set parameter TimeLimit to value 100000
Tour:  [0, 1, 9, 7, 8, 0, 2, 5, 6, 3, 4, 0]
Tour Time:  87.77882293568622
Left Demand:  tensor(0.)
Node Visits:  12
Games Finished:  0
####################################################################################################
Tour:  [0, 1, 9, 0, 6, 5, 0, 2, 0, 4, 8, 3, 7, 0]
Tour Time:  145.7299209100298
Left Demand:  tensor(-1.)
Node Visits:  14
Games Finished:  0
####################################################################################################
Set para

Tour:  [0, 1, 7, 0, 6, 0, 9, 8, 3, 2, 0, 4, 5, 0]
Tour Time:  96.44190764965641
Left Demand:  tensor(-1.)
Node Visits:  14
Games Finished:  0
####################################################################################################
Tour:  [0, 4, 2, 1, 0, 6, 9, 0, 8, 0, 5, 7, 3, 0]
Tour Time:  135.48598805515158
Left Demand:  tensor(-5.)
Node Visits:  14
Games Finished:  0
####################################################################################################
Set parameter TimeLimit to value 100000
Tour:  [0, 8, 3, 5, 4, 1, 6, 7, 9, 2, 0, 0]
Tour Time:  50.13669809130812
Left Demand:  0
Node Visits:  12
Games Finished:  0
####################################################################################################
Tour:  [0, 8, 3, 5, 4, 1, 0, 6, 7, 9, 2, 0]
Tour Time:  72.29637813056895
Left Demand:  tensor(-8.)
Node Visits:  12
Games Finished:  0
####################################################################################################
Set param

Tour:  [0, 4, 1, 2, 3, 8, 0, 9, 5, 6, 7, 0]
Tour Time:  63.417683282160176
Left Demand:  tensor(0.)
Node Visits:  12
Games Finished:  0
####################################################################################################
Tour:  [0, 4, 1, 2, 9, 5, 6, 0, 7, 8, 3, 0]
Tour Time:  64.79319325031851
Left Demand:  tensor(-4.)
Node Visits:  12
Games Finished:  0
####################################################################################################
Set parameter TimeLimit to value 100000
Tour:  [0, 2, 1, 6, 4, 5, 9, 3, 0, 7, 8, 0]
Tour Time:  85.77518692758663
Left Demand:  tensor(0.)
Node Visits:  12
Games Finished:  0
####################################################################################################
Tour:  [0, 7, 8, 2, 0, 1, 0, 6, 0, 4, 0, 3, 9, 0, 5, 0]
Tour Time:  234.89112703617195
Left Demand:  tensor(-5.)
Node Visits:  16
Games Finished:  0
####################################################################################################


In [14]:
print("MIP mean reward:", np.mean(results["mip"]["cost"]))
print("NN mean reward:", np.mean(results["nn"]["cost"]))

MIP mean reward: -145.35198181152344
NN mean reward: -284.83903350830076
